In [6]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove
from lib.state_machine import smart_contract_state_machine

In [7]:

statemachine = smart_contract_state_machine('Crowdsale')
state, stateOut = statemachine.add_state("state", z3.BitVecSort(2))
deposits, depositsOut = statemachine.add_state("deposits", z3.ArraySort(z3.BitVecSort(256), z3.BitVecSort(256)))
totalDeposits, totalDepositsOut = statemachine.add_state("totalDeposits", z3.BitVecSort(256))
raised, raisedOut = statemachine.add_state("raised", z3.BitVecSort(256))
aux_claimrefund, aux_claimrefundOut = statemachine.add_state("aux_claimrefund", z3.BoolSort())
aux_withdraw, aux_withdrawOut = statemachine.add_state("aux_withdraw", z3.BoolSort())

GOAL = 10000
CLOSETIME = 10000


OPEN = 0
SUCCESS = 1
REFUND = 2




In [8]:
p = z3.BitVec('p',256)
now = statemachine.nowOut

statemachine.set_init(z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,
    totalDeposits == 0,
    state == OPEN,
    aux_claimrefund == False,
    aux_withdraw == False
))


sender = z3.BitVec('sender',256)
value = z3.BitVec('amount',256)

statemachine.add_tr(
    tr_name = "invest",
    parameters = (value, sender),
    guard = z3.And(# now <= CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(raisedOut == raised + value,
                           depositsOut == z3.Update(deposits,sender,deposits[sender]+value),
                           totalDepositsOut == totalDeposits+value,
                        )
)

statemachine.add_tr(
    tr_name = "close_success",
    parameters = None,
    guard = z3.And(raised >= GOAL,
                   ),
    transfer_func = z3.And(
                           stateOut == SUCCESS,
                        )
)

statemachine.add_tr(
    tr_name = "close_refund",
    parameters = None,
    guard = z3.And(now > CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(stateOut == REFUND,
                        )
)

p = z3.BitVec('p',256)
statemachine.add_tr(
    tr_name = "claimrefund",
    parameters = (p, ),
    guard = z3.And(state == REFUND,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(depositsOut == z3.Update(deposits,p,0),
                           totalDepositsOut == totalDeposits - deposits[p],
                           aux_claimrefundOut == True,
                        )
)


statemachine.add_tr(
    tr_name = "withdraw",
    parameters = None,
    guard = z3.And(state == SUCCESS,
                   ),
    transfer_func = z3.And(totalDepositsOut == 0,
                           aux_withdrawOut == True,
                        )
)


In [9]:
# print(statemachine.now_state)
# statemachine.transfer("invest", amount == 100)
# print(statemachine.now_state)

positive_traces = []
positive_traces.append(
    [
        ('invest', now == 1, sender == 0x114514, value == 100),
        ('invest', now == 2, sender == 0x114515, value == 100),
        ('invest', now == 3, sender == 0x114516, value == 100),
        ('invest', now == 4, sender == 0x114517, value == 100),
        ('invest', now == 5, sender == 0x114518, value == 100),
        ('invest', now == 6, sender == 0x114519, value == 100),
        ('invest', now == 7, sender == 0x114520, value == 100),
    ]
)

positive_traces.append(
    [
        ('invest', now == 1, sender == 0x114514, value == 100),
        ('invest', now == 2, sender == 0x114515, value == GOAL),
        ('close_success', now == 3),
        ('withdraw', now == 4)
    ]
)

positive_traces.append(
    [
        ('invest', now == 1, sender == 0x114514, value == 100),
        ('invest', now == 2, sender == 0x114515, value == GOAL),
        ('close_success', now == CLOSETIME + 1),
        ('withdraw', now == CLOSETIME + 2)
    ]
)

positive_traces.append(
    [
        ('invest', now == 1, sender == 0x114514, value == 100),
        ('invest', now == 2, sender == 0x114515, value == 100),
        ('close_refund', now == CLOSETIME + 1),
        ('claimrefund', now == CLOSETIME + 2, p == 0x114514)
    ]
)

negative_traces = []
negative_traces.append(
    [
        ('close_refund', now == CLOSETIME + 1),
        ('claimrefund', p == 0x114514, now == CLOSETIME + 2),
        ('invest', value == GOAL+1, now == CLOSETIME + 3),
        ('close_success', now == CLOSETIME + 4),
        ('withdraw', now == CLOSETIME + 5),
    ]
)
r2 = z3.Not(z3.And(aux_withdraw, aux_claimrefund))



test_ntrace = negative_traces[0]
test_ptrace = positive_traces[0]

# print(statemachine.simulate(test_ntrace, show_log=True))



test_ntrace = statemachine.bmc(z3.Not(r2))
print(test_ntrace)

statemachine.add_guard('invest', now <= CLOSETIME)
print(statemachine.bmc(z3.Not(r2)))
t = [('claimrefund', 2 == now), ('withdraw', 3 == now)]

print(statemachine.simulate(test_ntrace, show_log=False))



[('close_refund', 10001 == now'), ('claimrefund', 10004 == now', 6901746346790563787434755862277025452451108972170386555162524223799296 ==
p), ('invest', 10005 == now', 16114218197058785108136390618034395594935769378727536403400168956387942992068 ==
amount, 113078212145816597093331040047546785012958969400039613319782796882727665664 ==
sender), ('close_success', 10007 == now'), ('withdraw', 10012 == now')]
None
reject


In [10]:
statemachine.clear_guards()

possible_guards = [
    state == OPEN, 
    state == SUCCESS, 
    state == REFUND, 
    now > CLOSETIME, 
    now < CLOSETIME, 
    raised >= GOAL, 
    raised < GOAL, 
]
# for p in positive_traces:
#     print(statemachine.simulate(p, show_log=False))

# res = statemachine.synthesis_guard(possible_guards, test_ntrace, positive_traces)
# print(res)

iter = 0
# while True:
while iter < 100:
    print("iter", iter)
    iter += 1
    ntrace = statemachine.bmc(z3.Not(r2))
    if ntrace == None:
        print("no more counter example")
        break
    print("find counter example:")
    print(ntrace)
    result_guard = statemachine.synthesize_one_guard(possible_guards, ntrace, positive_traces)
    tr, g = result_guard[0]
    print("synthesized guard:")
    print(result_guard)
    statemachine.add_guard(tr, g)
    print()

print("Final guards:")
print(statemachine.condition_guards)




iter 0


find counter example:
[('withdraw', 2 == now'), ('claimrefund', 3 == now', 1 == p)]
synthesized guard:
[['claimrefund', state == 2], ['claimrefund', now' > 10000], ['withdraw', state == 1], ['withdraw', raised >= 10000]]

iter 1
find counter example:
[('close_refund', 1 == now'), ('withdraw', 2 == now'), ('claimrefund', 3 == now', 0 == p)]
synthesized guard:
[['close_refund', now' > 10000], ['claimrefund', now' > 10000], ['withdraw', state == 1], ['withdraw', raised >= 10000]]

iter 2
find counter example:
[('close_refund', 10001 == now'), ('withdraw', 10002 == now'), ('claimrefund', 10003 == now', 0 == p)]
synthesized guard:
[['withdraw', state == 1], ['withdraw', raised >= 10000]]

iter 3
find counter example:
[('close_success', 10032 == now'), ('withdraw', 12289 == now'), ('close_refund', 12290 == now'), ('claimrefund', 14099 == now', 0 == p)]
synthesized guard:
[['close_success', raised >= 10000], ['close_refund', state == 0], ['withdraw', raised >= 10000]]

iter 4
find counter exa